In [11]:
import tensorflow as tf

In [12]:


x1= tf.constant(5)
x2 = tf.constant(6)

res1 = tf.add(x1,x2)
res2 = tf.subtract(x1,x2)
res3 = tf.multiply(x1,x2)
res4 = tf.divide(x1,x2)


sess = tf.Session()
print(sess.run(res1))
print(sess.run(res2))
print(sess.run(res3))
print(sess.run(res4))

sess.close()

11
-1
30
0.833333333333


## Another way of starting a session which does not need a separate close() call is as: ##

In [70]:
with tf.Session() as sess:
    out1 = sess.run(res1)
    out2 = sess.run(res2)
    out3 = sess.run(res3)
    out4 = sess.run(res4)
    
    print(out1)
    print(out2)
    print(out3)
    print(out4)
    

11
-1
30
0.833333333333


Checking the variable values after closing the session()

In [71]:
print(out1)
print(out2)
print(out3)
print(out4)

11
-1
30
0.833333333333


# Creating a neural network using MNIST dataset #

Basic idea : i/p +weights> hidden layer 1 (activation) +weights> hidden layer 2 (activation) + weights> output layer 

compare output to intended output > cost function
optimization function (optimizer): to minimize cost (Adam optimizer, SGD, Adagrad)

backpropogation

(feedforward + backpropogation = epoch)

In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True) #one hot is useful for multiclass classification

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [27]:
n_nodes_hl1 = 500 #no of nodes in hl1
n_nodes_hl2 = 600 #no of nodes in hl2
n_nodes_hl3 = 400 #no of nodes in hl3

n_classes = 10 #no of classes [0-9] 
batch_size = 100

In [28]:
#height x width
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

In [40]:
##################### DEFINING THE COMPUTATIONAL GRAPH ##############################

#(input *weights)+biases
def neural_network_model(data):
    
    hidden_layer_1 = {'Weights':tf.Variable(tf.random_normal([784,n_nodes_hl1])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_layer_2 = {'Weights':tf.Variable(tf.random_normal([n_nodes_hl1,n_nodes_hl2])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_layer_3 = {'Weights':tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    output_layer = {'Weights':tf.Variable(tf.random_normal([n_nodes_hl3,n_classes])),
                     'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    l1 = tf.add(tf.matmul(data, hidden_layer_1['Weights']), hidden_layer_1['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_layer_2['Weights']), hidden_layer_2['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_layer_3['Weights']), hidden_layer_3['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.matmul(l3, output_layer['Weights']), output_layer['biases'])
 
    return output

In [49]:

################ TRAINING THE NEURAL NETWORK ########################
def train_network(x): 
    
    prediction = neural_network_model(x)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = prediction, labels = y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    #feed_forwards + backprops
    hm_epochs = 10
    
    with tf.Session() as sess: 
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss= 0 
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                _ , c= sess.run([optimizer,cost], feed_dict = {x:epoch_x, y:epoch_y})
                epoch_loss += c
            print("Epoch",epoch, 'completed out of',hm_epochs,'loss: ', epoch_loss)
        correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy: ', accuracy.eval({x:mnist.test.images, y: mnist.test.labels}))
            

Epoch 0 completed out of 10 loss:  1683882.45468
Epoch 1 completed out of 10 loss:  388390.685524
Epoch 2 completed out of 10 loss:  208075.860565
Epoch 3 completed out of 10 loss:  120530.435566
Epoch 4 completed out of 10 loss:  73695.870473
Epoch 5 completed out of 10 loss:  45921.7156533
Epoch 6 completed out of 10 loss:  32144.9446572
Epoch 7 completed out of 10 loss:  25270.3447558
Epoch 8 completed out of 10 loss:  21266.9249277
Epoch 9 completed out of 10 loss:  17575.8923841
Accuracy:  0.9467
